In [1]:

import os
from tqdm import tqdm
import cv2
import torch
import numpy as np

In [2]:
gt_path = '../testdata/labels/'
preds_path = '../testdata/preds/'

# get all imgs'name, remove the extension
images_folder1 = [f for f in os.listdir(gt_path) if
                  os.path.isfile(os.path.join(gt_path, f)) and f.lower().endswith(
                      ('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.gif'))]
images_names = [os.path.splitext(f)[0] for f in images_folder1]
tbar = tqdm(images_names)



  0%|          | 0/21 [00:00<?, ?it/s]

# Target Center Metric

In [3]:
from sosmetrics.metrics import TargetPrecisionRecallF1
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = TargetPrecisionRecallF1(
    dis_thrs=[1, 10],
    conf_thr=0.5,
    match_alg='forloop',
    second_match='mask_plus'
    
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()

Reading image_name=0000:   0%|          | 0/21 [00:01<?, ?it/s]Exception in thread Thread-6:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\target_pre_rec_f1.py", line 96, in evaluate_worker
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    raise ValueError("Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']")
ValueError: Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_pl

Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPrecisionRecallF1.update() took 0.01s each time.
+---------+----+----+----+------------------+---------------+----------------+
| Dis-Thr | TP | FP | FN | target_Precision | target_Recall | target_F1score |
+---------+----+----+----+------------------+---------------+----------------+
|   1.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   2.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   3.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   4.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   5.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   6.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   7.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   8.0   | 0  | 0  | 0  |     0.00000

Exception in thread Thread-78:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-69:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-72:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
Exception in thread Thread-83:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_c

TargetPrecisionRecallF1.update() took 0.01s each time.
+---------+----+----+----+------------------+---------------+----------------+
| Dis-Thr | TP | FP | FN | target_Precision | target_Recall | target_F1score |
+---------+----+----+----+------------------+---------------+----------------+
|   1.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   2.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   3.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   4.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   5.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   6.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   7.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   8.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   9.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000    

        raise ValueError("Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']")
ValueError: Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\target_pre_rec_f1.py", line 96, in evaluate_worker
distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarys

TargetPrecisionRecallF1.update() took 0.02s each time.
+---------+----+----+----+------------------+---------------+----------------+
| Dis-Thr | TP | FP | FN | target_Precision | target_Recall | target_F1score |
+---------+----+----+----+------------------+---------------+----------------+
|   1.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   2.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   3.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   4.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   5.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   6.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   7.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   8.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000     |
|   9.0   | 0  | 0  | 0  |     0.00000      |    0.00000    |    0.00000    

(array([7.29112202e-304, 7.29112202e-304, 8.32155212e-317, 2.13031734e-314,
        3.25060610e-319, 2.13031734e-314, 7.74860419e-304, 7.74681715e-304,
        1.26974871e-321, 3.25055670e-319]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [4]:
Metric.table

,dis_thrs,TP,FP,FN,Precision,Recall,F1
0,1.0,0.0,0.0,0.0,7.291122e-304,0.0,0.0
1,2.0,0.0,0.0,0.0,7.291122e-304,0.0,0.0
2,3.0,0.0,0.0,0.0,8.321552e-317,0.0,0.0
3,4.0,0.0,0.0,0.0,2.130317e-314,0.0,0.0
4,5.0,0.0,0.0,0.0,3.250606e-319,0.0,0.0
5,6.0,0.0,0.0,0.0,2.130317e-314,0.0,0.0
6,7.0,0.0,0.0,0.0,7.748604e-304,0.0,0.0
7,8.0,0.0,0.0,0.0,7.746817e-304,0.0,0.0
8,9.0,0.0,0.0,0.0,1.269749e-321,0.0,0.0
9,10.0,0.0,0.0,0.0,3.250557e-319,0.0,0.0


## Target Average Precision


In [5]:
from sosmetrics.metrics import TargetAveragePrecision
Metric = TargetAveragePrecision(
    dis_thrs=[1, 10],
    conf_thrs=10,
    match_alg='forloop',
    second_match='mask_plus',
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Exception in thread Thread-153:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\target_ap.py", line 100, in evaluate_worker
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    raise ValueError("Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']")
ValueError: Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']
Exception in thread Thread-154:
Traceback (most recent call las

Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetAveragePrecision.update() took 0.01s each time.
+---------+-----+---------+---------+---------+---------+---------+---------+---------+---------+------+
| Dis—Thr | 1.0 |   2.0   |   3.0   |   4.0   |   5.0   |   6.0   |   7.0   |   8.0   |   9.0   | 10.0 |
+---------+-----+---------+---------+---------+---------+---------+---------+---------+---------+------+
|    AP   | nan | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | nan  |
+---------+-----+---------+---------+---------+---------+---------+---------+---------+---------+------+
TargetAveragePrecision(match_alg=forloop)
Test image of list [hwc, hwc, ...]
TargetAveragePrecision.update() took 0.01s each time.
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Dis—Thr |   1.0   |   2.0   |   3.0   |   4.0   |   5.0   |

Exception in thread Thread-217:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
Exception in thread Thread-216:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    _threading_Thread_run(self)
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\target_ap.py", line 100, in evaluate_worker
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_meth

TargetAveragePrecision.update() took 0.02s each time.
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Dis—Thr |   1.0   |   2.0   |   3.0   |   4.0   |   5.0   |   6.0   |   7.0   |   8.0   |   9.0   |   10.0  |
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|    AP   | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 |
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
TargetAveragePrecision(match_alg=forloop)
Test image of np.array, bhwc
TargetAveragePrecision.update() took 0.02s each time.
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Dis—Thr |   1.0   |   2.0   |   3.0   |   4.0   |   5.0   |   6.0   |   7.0   |   8.0   |   9.0   |   10.0  |
+---------+---------+---------+------

    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 870, in run
Exception in thread Thread-260:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\target_ap.py", line 100, in evaluate_worker
Exception in thread Thread-259:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-261:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    distances = s

TargetAveragePrecision.update() took 0.02s each time.
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Dis—Thr |   1.0   |   2.0   |   3.0   |   4.0   |   5.0   |   6.0   |   7.0   |   8.0   |   9.0   |   10.0  |
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|    AP   | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 | -0.0000 |
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
TargetAveragePrecision(match_alg=forloop)
Test image of list,  [chw, chw, ...]
TargetAveragePrecision.update() took 0.02s each time.
+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
| Dis—Thr |   1.0   |   2.0   |   3.0   |   4.0   |   5.0   |   6.0   |   7.0   |   8.0   |   9.0   |   10.0  |
+---------+---------+--------

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

In [6]:
Metric.table

,0,1,2,3,4,5,6,7,8,9
Dis-Thr,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
AP,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0


# PD_FA Metric

In [7]:
# For test multiple format of input.
from sosmetrics.metrics import TargetPdPixelFa


gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []

Metric = TargetPdPixelFa(
    dis_thrs=[1, 10],
    conf_thr=0.5,
    match_alg='forloop',
        second_match='mask_plus'
    # debug = True
    )
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png") 
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format
    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Exception in thread Thread-300:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\hybrid_pd_fa.py", line 110, in evaluate_worker
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    raise ValueError("Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']")
ValueError: Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']
Exception in thread Thread-301:
Traceback (most recent call 

Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
TargetPdPixelFa.update() took 0.01s each time.
+-----------+----+----+----+----+-----------+--------------+
| Threshold | TD | AT | FD | NP | target_Pd |   pixel_Fa   |
+-----------+----+----+----+----+-----------+--------------+
|    1.0    | 0  | 0  | 0  | 0  |  0.00000  | 5.45353e-312 |
|    2.0    | 0  | 0  | 0  | 0  |  0.00000  | 5.45353e-312 |
|    3.0    | 0  | 0  | 0  | 0  |  0.00000  | 1.39612e-309 |
|    4.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.74860e-304 |
|    5.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.74860e-304 |
|    6.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.74860e-304 |
|    7.0    | 0  | 0  | 0  | 0  |  0.00000  | 5.43239e-312 |
|    8.0    | 0  | 0  | 0  | 0  |  0.00000  | 3.25061e-319 |
|    9.0    | 0  | 0  | 0  | 0  |  0.00000  | 3.25061e-319 |
|    10.0   | 0  | 0  | 0  | 0  |  0.00000  | 3.25061e-319 |
+-----------+----+----+----+----+----------

    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
Exception in thread Thread-369:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
Exception in thread Thread-371:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-package

TargetPdPixelFa.update() took 0.01s each time.
+-----------+----+----+----+----+-----------+--------------+
| Threshold | TD | AT | FD | NP | target_Pd |   pixel_Fa   |
+-----------+----+----+----+----+-----------+--------------+
|    1.0    | 0  | 0  | 0  | 0  |  0.00000  | 5.45353e-312 |
|    2.0    | 0  | 0  | 0  | 0  |  0.00000  | 5.45353e-312 |
|    3.0    | 0  | 0  | 0  | 0  |  0.00000  | 1.39612e-309 |
|    4.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.74860e-304 |
|    5.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.74860e-304 |
|    6.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.74860e-304 |
|    7.0    | 0  | 0  | 0  | 0  |  0.00000  | 5.43239e-312 |
|    8.0    | 0  | 0  | 0  | 0  |  0.00000  | 3.25061e-319 |
|    9.0    | 0  | 0  | 0  | 0  |  0.00000  | 3.25061e-319 |
|    10.0   | 0  | 0  | 0  | 0  |  0.00000  | 3.25061e-319 |
+-----------+----+----+----+----+-----------+--------------+
TargetPdPixelFa(conf_thr=[0.5], match_alg=forloop)
Test image of np.array, bhwc
TargetPdPixelFa.upd

    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    raise ValueError("Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']")
ValueError: Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']
    raise ValueError("Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']")
ValueError: Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']
Exception in thread Thread-428:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda

TargetPdPixelFa.update() took 0.02s each time.
+-----------+----+----+----+----+-----------+--------------+
| Threshold | TD | AT | FD | NP | target_Pd |   pixel_Fa   |
+-----------+----+----+----+----+-----------+--------------+
|    1.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.29112e-304 |
|    2.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.29112e-304 |
|    3.0    | 0  | 0  | 0  | 0  |  0.00000  | 8.32155e-317 |
|    4.0    | 0  | 0  | 0  | 0  |  0.00000  | 5.45361e-312 |
|    5.0    | 0  | 0  | 0  | 0  |  0.00000  | 2.12203e-314 |
|    6.0    | 0  | 0  | 0  | 0  |  0.00000  | 5.45361e-312 |
|    7.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.74860e-304 |
|    8.0    | 0  | 0  | 0  | 0  |  0.00000  | 7.74682e-304 |
|    9.0    | 0  | 0  | 0  | 0  |  0.00000  | 1.26975e-321 |
|    10.0   | 0  | 0  | 0  | 0  |  0.00000  | 8.32155e-317 |
+-----------+----+----+----+----+-----------+--------------+
TargetPdPixelFa(conf_thr=[0.5], match_alg=forloop)
Test image of list,  [chw, chw, ...]
TargetPdPix

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([7.29112202e-304, 7.29112202e-304, 8.32155212e-317, 5.45361240e-312,
        2.12202830e-314, 5.45361240e-312, 7.74860419e-304, 7.74681715e-304,
        1.26974871e-321, 8.32155163e-317]))

In [8]:
Metric.table

,dis_thrs,TD,AT,FD,NP,target_Pd,pixel_Fa
0,1.0,0.0,0.0,0.0,0.0,0.0,7.291122e-304
1,2.0,0.0,0.0,0.0,0.0,0.0,7.291122e-304
2,3.0,0.0,0.0,0.0,0.0,0.0,8.321552e-317
3,4.0,0.0,0.0,0.0,0.0,0.0,5.453612e-312
4,5.0,0.0,0.0,0.0,0.0,0.0,2.122028e-314
5,6.0,0.0,0.0,0.0,0.0,0.0,5.453612e-312
6,7.0,0.0,0.0,0.0,0.0,0.0,7.748604e-304
7,8.0,0.0,0.0,0.0,0.0,0.0,7.746817e-304
8,9.0,0.0,0.0,0.0,0.0,0.0,1.269749e-321
9,10.0,0.0,0.0,0.0,0.0,0.0,8.321552e-317


# PixelNormalizeIoU

In [9]:
from sosmetrics.metrics import PixelNormalizedIoU
Metric = PixelNormalizedIoU(
    conf_thr=0.5,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of list [hwc, hwc, ...]
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image_path of list, [img_path, img_path, ...]
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of np.array, bhwc
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
Test image of tensor, bchw
PixelNormalizedIoU.update() took 0.00s each time.
+----------+
| nIoU-0.5 |
+----------+
|  0.7178  |
+----------+
PixelNormalizedIoU(conf_thr=0.5)
T

0.7178133052159646

In [10]:
Metric.table

,nIoU
0,0.717813


## HybridNormalizedIoU

In [11]:
from sosmetrics.metrics import HybridNormalizedIoU
Metric = HybridNormalizedIoU(
    conf_thr=0.5,
    dis_thrs = [1,10],
    match_alg='forloop',
    second_match='mask_plus'
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Exception in thread Thread-447:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\hybrid_normalized_iou.py", line 73, in evaluate_worker
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
    raise ValueError("Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']")
ValueError: Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']
Exception in thread Thread-448:
Traceback (most rece

Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
HybridNormalizedIoU.update() took 0.01s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| nIoU-0.5 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
HybridNormalizedIoU(conf_thr=0.5)
Test image of list [hwc, hwc, ...]
HybridNormalizedIoU.update() took 0.01s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   | 

    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    self.run()
  File "c:\Users\songj\miniconda3\envs\sos\lib\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\hybrid_normalized_iou.py", line 73, in evaluate_worker
Exception in thread Thread-513:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-515:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    _threading_Thread_run(self)
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 870, in run
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second

HybridNormalizedIoU.update() took 0.01s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| nIoU-0.5 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
HybridNormalizedIoU(conf_thr=0.5)
Test image of np.array, bhwc
HybridNormalizedIoU.update() took 0.01s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| nIoU-0.5 

Exception in thread Thread-571:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "d:\project\binarysosmetrics\sosmetrics\metrics\hybrid_normalized_iou.py", line 73, in evaluate_worker
    _threading_Thread_run(self)
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 870, in run
    raise ValueError("Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']")
ValueError: Invalid match mode, match_mode should be one of ['mask', 'bbox', 'mask_plus', 'bbox_plus']
    distances = second_match_method(distances, mask_iou, bbox_iou,
  File "d:\project\binarysosmetrics\sosmetrics\metrics\utils\target_infos.py", line 134, in second_match_method
Exception in thread Thread-570:
Traceback (most recent call last):
  File "c:\Users\songj\miniconda3\envs\sos\lib\threading.py", line 932, in _bootstrap_inner
    _threading_Thr

HybridNormalizedIoU.update() took 0.02s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| nIoU-0.5 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 | 0.0000 |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
HybridNormalizedIoU(conf_thr=0.5)
Test image of list,  [chw, chw, ...]
HybridNormalizedIoU.update() took 0.02s each time.
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| Dis—Thr  |  1.0   |  2.0   |  3.0   |  4.0   |  5.0   |  6.0   |  7.0   |  8.0   |  9.0   |  10.0  |
+----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+
| n

array([7.29112202e-304, 7.29112202e-304, 8.32155212e-317, 2.13031734e-314,
       3.25060610e-319, 2.13031734e-314, 7.74860419e-304, 7.74681715e-304,
       1.26974871e-321, 3.25055670e-319])

In [12]:
Metric.table

,0,1,2,3,4,5,6,7,8,9
Dis-Thr,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00,5.000000e+00,6.000000e+00,7.000000e+00,8.000000e+00,9.000000e+00,1.000000e+01
nIoU,7.291122e-304,7.291122e-304,8.321552e-317,2.130317e-314,3.250606e-319,2.130317e-314,7.748604e-304,7.746817e-304,1.269749e-321,3.250557e-319


# Pixel ROC Precision Recall Metric

In [13]:
from sosmetrics.metrics import PixelROCPrecisionRecall
Metric = PixelROCPrecisionRecall(
    conf_thrs=30,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelROCPrecisionRecall.update() took 0.06s each time.
+--------------------+----------------------+-------------------------------------+
|      AUC_ROC       | AUC_PR(AUC function) | AP(BinaryAveragePrecision function) |
+--------------------+----------------------+-------------------------------------+
| 0.9427307844161987 |  0.8806460499763489  |              0.8124518              |
+--------------------+----------------------+-------------------------------------+
PixelROCPrecisionRecall
Test image of list [hwc, hwc, ...]
PixelROCPrecisionRecall.update() took 0.08s each time.
+--------------------+----------------------+-------------------------------------+
|      AUC_ROC       | AUC_PR(AUC function) | AP(BinaryAveragePrecision function) |
+--------------------+----------------------+-------------------------------------+
| 0.9427307844161987 |  0.8806460499763489

(0.9427307844161987,
 0.8806460499763489,
 array([9.45113607e-06, 6.25229004e-05, 6.90659945e-05, 7.19740347e-05,
        7.63361022e-05, 7.77901223e-05, 7.85171287e-05, 7.99711488e-05,
        8.14251762e-05, 8.36062027e-05, 8.57872365e-05, 8.72412566e-05,
        9.16033168e-05, 9.52383707e-05, 9.66923908e-05, 9.66923908e-05,
        9.74194045e-05, 9.81464109e-05, 9.96004383e-05, 1.00327445e-04,
        1.02508478e-04, 1.05416519e-04, 1.06870539e-04, 1.08324559e-04,
        1.10505593e-04, 1.13413633e-04, 1.16321673e-04, 1.25772814e-04,
        1.36677976e-04, 1.00000000e+00], dtype=float32),
 array([0.45789474, 0.79868424, 0.8144737 , 0.8210526 , 0.8276316 ,
        0.83157897, 0.8342105 , 0.8381579 , 0.8407895 , 0.8407895 ,
        0.84210527, 0.84605265, 0.84605265, 0.8473684 , 0.8486842 ,
        0.85      , 0.85263157, 0.85394734, 0.85394734, 0.8578947 ,
        0.8605263 , 0.8605263 , 0.8618421 , 0.86315787, 0.8644737 ,
        0.86973685, 0.86973685, 0.8723684 , 0.8855263 , 1

In [14]:
Metric.table

,AUC_ROC,AUC_PR,AP
0,0.942731,0.880646,0.812452


# Piexl Precision Recall F1 IoU 

In [15]:
from sosmetrics.metrics import PixelPrecisionRecallF1IoU
Metric = PixelPrecisionRecallF1IoU(
    conf_thr=0.5,
    # debug = True
    )
# For test multiple format of input.
gt_img_list = []
pred_img_list = [] 

gt_img_list_chw = []
pred_img_list_chw = []

gt_img_paths = []
pred_img_paths = []
for image_name in tbar:
    tbar.set_description(f"Reading image_name={image_name}")
    gt_image_path = os.path.join(gt_path, f"{image_name}.png")  # 假设所有图片都是PNG格式，或者你可以根据实际情况修改
    pred_image_path = os.path.join(preds_path, f"{image_name}.png") 

    gt_img = cv2.imread(gt_image_path)
    pred_img = cv2.imread(pred_image_path)

    # for test [chw, chw, ...] format
    gt_img_list_chw.append(gt_img.transpose(2,1,0))
    pred_img_list_chw.append(pred_img.transpose(2,1,0))

    # for test [hwc, hwc] format
    gt_img_list.append(gt_img)
    pred_img_list.append(pred_img)

    # for test [path, path] format

    gt_img_paths.append(gt_image_path)
    pred_img_paths.append(pred_image_path)

    # for test single img or img path
    Metric.update(gt_img, pred_img)
    Metric.update(gt_image_path, pred_image_path)

print("Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.")
Metric.get()
Metric.reset()


print("Test image of list [hwc, hwc, ...]")
Metric.update(gt_img_list, pred_img_list) 
Metric.get()
Metric.reset()

print("Test image_path of list, [img_path, img_path, ...]")
Metric.update(gt_img_paths, pred_img_paths)
Metric.get()
Metric.reset()

print("Test image of np.array, bhwc")
Metric.update(np.stack(gt_img_list), np.stack(pred_img_list)) 
Metric.get()
Metric.reset()


print("Test image of tensor, bchw")
Metric.update(torch.from_numpy(np.stack(gt_img_list_chw)), torch.from_numpy(np.stack(pred_img_list_chw))) 
Metric.get()
Metric.reset()

print("Test image of list,  [chw, chw, ...]")
Metric.update(gt_img_list_chw, pred_img_list_chw) 
Metric.get()


Simultaneous test single image path and single image, (TP, FN, FP) should be two times as big as the following test.
PixelPrecisionRecallF1IoU.update() took 0.01s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall-0.5 | F1-0.5 | IOU-0.5 |
+---------------+------------+--------+---------+
|     0.8293    |   0.8500   | 0.8395 | 0.72340 |
+---------------+------------+--------+---------+
PixelPrecisionRecallF1IoU
Test image of list [hwc, hwc, ...]
PixelPrecisionRecallF1IoU.update() took 0.03s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall-0.5 | F1-0.5 | IOU-0.5 |
+---------------+------------+--------+---------+
|     0.8293    |   0.8500   | 0.8395 | 0.72340 |
+---------------+------------+--------+---------+
PixelPrecisionRecallF1IoU
Test image_path of list, [img_path, img_path, ...]
PixelPrecisionRecallF1IoU.update() took 0.03s each time.
+---------------+------------+--------+---------+
| Precision-0.5 | Recall

(array([0.82926829]), array([0.85]), array([0.83950617]), array([0.72340426]))

In [16]:
Metric.table

,Precision-0.5,Recall-0.5,F1-0.5,IOU-0.5
0,0.829268,0.85,0.839506,0.723404
